# Causal Discovery with WhyNot

WhyNot provides tools to automatically construct *the causal graph* associated with runs of the dynamical system simulators.
Building off of work in [automatic differentiation](https://github.com/HIPS/autograd), WhyNot traces the evoluation of the state variables
during simulation and builds up the corresponding causal graph. This allows the developer to write complicated simulators using raw Python
and Numpy and then automatically extract the graph of the dynamics in a way that is more flexible and less error-prone than tracking the dynamics by hand.


In this notebook, we leverage these tools to test causal discovery algorithms. In particular, using the HIV simulator, we generate a dataset
of rollouts of the dynamics, as well as the associated causal graph. We then evaluate the performance of the IC* (Inductive Causation with latent
variables) algorithm from Pearl, 2000. We use the independence tests and IC implementation provided by the [causality](https://www.github.com/akelleh/causality) package.


**Note**: This feature is still experimental, and there are likely a few rough edges. Only the HIV, Lotka-Volterra, and Opioid Epidemic simulators support causal graph generation at present. We are rapidly working on extending this feature to all of the dynamical system models.

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import whynot as wn
import numpy as np
import pandas as pd

from causal_search import IC, undirected_f1, directed_f1
from causality.inference.independence_tests import RobustRegressionTest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Sanity Checks
First, we sanity check the implementation of the IC algorithm on a simple data example provided by the [causality](https://github.com/akelleh/causality) package.

In [2]:
# Toy dataset generation
samples = 200
x1 = np.random.normal(size=samples)
x2 = x1 + np.random.normal(size=samples)
x3 = x1 + np.random.normal(size=samples)
x4 = x2 + x3 + np.random.normal(size=samples)
x5 = x4 + np.random.normal(size=samples)

# load the data into a dataframe:
X = pd.DataFrame({'x1' : x1, 'x2' : x2, 'x3' : x3, 'x4' : x4, 'x5' : x5})

# define the variable types: 'c' is 'continuous'.  The variables defined here
# are the ones the search is performed over  -- NOT all the variables defined
# in the data frame.
variable_types = {'x1' : 'c', 'x2' : 'c', 'x3' : 'c', 'x4' : 'c', 'x5' : 'c'}

# run the search
ic_algorithm = IC(RobustRegressionTest)
graph = ic_algorithm.search(X, variable_types)

In [3]:
# View the edges estimated by the IC algorithm
for edge, data in graph.edges.items():
    print(edge, data)

('x1', 'x2') {'marked': False, 'arrows': []}
('x2', 'x4') {'marked': False, 'arrows': ['x4']}
('x3', 'x4') {'marked': False, 'arrows': ['x4']}
('x4', 'x5') {'marked': True, 'arrows': ['x5']}


## Learning the Dynamics of the HIV Simulator

Now, we run an experiment to discover the causal structure of the dynamics for the HIV simulator. Rather than try to learn the structure
for the entire unrolled dynamics, we instead focus on learning the causal structure of the dynamics *for a single time step*. There are 6 states
in the simulator, given a state $x_0 \in \mathbb{R}^6$, the dynamics evolve $x_{1} = f(x_0) \in \mathbb{R}^6$, and we wish to uncover how each
component of $x_{1}$, e.g. "infected CD4+ T-lymphocytes", depends on the components of $x_0$, e.g. "infected macrophages." Hence, in this experiment,
there are 12 nodes (one for each component of the state at time steps 0 and 1), and 20 directed edges between them determined by tracing the simulator execution.

In [4]:
# Set up the initial simulator data. Each sample is an IID draw from a state distribution.
num_samples = 500

def sample_initial_state():
    """Sample initial state by randomly perturbing the default state."""
    state = wn.hiv.State().values()
    perturbed = state * np.random.uniform(low=0.95, high=1.05, size=6)
    return wn.hiv.State(*perturbed)

initial_states = [sample_initial_state() for _ in range(num_samples)]

# For illustrative purposes, only run the dynamics for a single time-step
config = wn.hiv.Config(delta_t=1.0, end_time=1)

In [5]:
# Rollout the simulator from each of the initial conditions
runs = [wn.hiv.simulate(init_state, config) for init_state in initial_states]

In [6]:
# Use WhyNot, to extract the underlying causal graph for this process
true_graph = wn.causal_graphs.build_dynamics_graph(wn.hiv, runs, config)

print("Number of nodes: {}, Number of edges: {}\n".format(len(true_graph.nodes), len(true_graph.edges)))
print(true_graph.nodes)

Number of nodes: 12, Number of edges: 20

['uninfected_T1_0.0', 'infected_T1_0.0', 'uninfected_T2_0.0', 'infected_T2_0.0', 'free_virus_0.0', 'immune_response_0.0', 'uninfected_T1_1.0', 'infected_T1_1.0', 'uninfected_T2_1.0', 'infected_T2_1.0', 'free_virus_1.0', 'immune_response_1.0']


In [7]:
# Generate a dataset consisting of all of the simulator covariates, unrolled over time
def flatten(run):
    """Flatten the covariates into a single long observation"""
    return np.concatenate([state.values() for state in run.states])

data = np.array([flatten(run) for run in runs]) 
columns = [f"{name}_{time}" for time, name in itertools.product(runs[0].times, wn.hiv.State.variable_names())]
df_hiv = pd.DataFrame(data, columns=columns)
df_hiv.head()

,uninfected_T1_0.0,infected_T1_0.0,uninfected_T2_0.0,infected_T2_0.0,free_virus_0.0,immune_response_0.0,uninfected_T1_1.0,infected_T1_1.0,uninfected_T2_1.0,infected_T2_1.0,free_virus_1.0,immune_response_1.0
0,9.703097e+05,0.000098,3347.233742,0.000101,1.040125,10.051451,9.706034e+05,1.439246,3345.015988,0.620209,8.106310,10.059732
1,1.001275e+06,0.000100,3147.733603,0.000098,1.003686,9.835796,1.001260e+06,1.451263,3147.560211,0.570337,7.948844,9.864060
2,9.685274e+05,0.000100,3110.889795,0.000098,0.996485,9.542907,9.688391e+05,1.288066,3111.143843,0.517111,7.145058,9.597626
3,1.001273e+06,0.000098,3142.639336,0.000099,0.986170,9.871409,1.001259e+06,1.424074,3142.530275,0.558756,7.797275,9.896090
4,1.004812e+06,0.000096,3334.378382,0.000096,1.023971,9.579640,1.004762e+06,1.571770,3332.253451,0.651737,8.701689,9.632993


In [56]:
# Run the search algorithm (this takes a while)
# All variables are continuous
variable_types = {column: 'c' for column in columns}
ic_algorithm = IC(RobustRegressionTest)
estimated_graph = ic_algorithm.search(df_hiv, variable_types)

In [57]:
for edge, data in estimated_graph.edges.items():
    print(edge, data)

('uninfected_T1_0.0', 'uninfected_T1_1.0') {'marked': False, 'arrows': []}
('uninfected_T1_0.0', 'infected_T1_1.0') {'marked': False, 'arrows': ['infected_T1_1.0']}
('uninfected_T2_0.0', 'immune_response_0.0') {'marked': False, 'arrows': []}
('uninfected_T2_0.0', 'uninfected_T2_1.0') {'marked': False, 'arrows': ['uninfected_T2_1.0']}
('uninfected_T2_0.0', 'immune_response_1.0') {'marked': False, 'arrows': []}
('immune_response_0.0', 'uninfected_T2_1.0') {'marked': False, 'arrows': ['uninfected_T2_1.0']}
('immune_response_0.0', 'immune_response_1.0') {'marked': False, 'arrows': []}
('uninfected_T1_1.0', 'infected_T1_1.0') {'marked': False, 'arrows': ['infected_T1_1.0']}
('infected_T1_1.0', 'infected_T2_1.0') {'marked': False, 'arrows': ['infected_T1_1.0', 'infected_T1_1.0', 'infected_T2_1.0']}
('infected_T1_1.0', 'free_virus_1.0') {'marked': True, 'arrows': ['free_virus_1.0', 'free_virus_1.0']}
('uninfected_T2_1.0', 'free_virus_1.0') {'marked': False, 'arrows': ['uninfected_T2_1.0', 'un

In [58]:
# How well did the causal structure learning algorithm perform?
print("Original Graph: {} edges, Estimated Graph: {} edges".format(len(true_graph.edges), len(estimated_graph.edges)))
print("Undirected Edge F1 Score: {:.2f}".format(undirected_f1(true_graph, estimated_graph)))
print("Directed Edge F1 Score: {:.2f}".format(directed_f1(true_graph, estimated_graph)))

Original Graph: 20 edges, Estimated Graph: 13 edges
Undirected Edge F1 Score: 0.24
Directed Edge F1 Score: 0.14
